In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [18]:
no_pages = 2

def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get('https://www.amazon.in/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []
    for d in soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
        #print(d)
        name = d.find('span', attrs={'class':'zg-text-center-align'})
        n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        author = d.find('a', attrs={'class':'a-size-small a-link-child'})
        rating = d.find('span', attrs={'class':'a-icon-alt'})
        users_rated = d.find('a', attrs={'class':'a-size-small a-link-normal'})
        price = d.find('span', attrs={'class':'p13n-sc-price'})

        all1=[]

        if name is not None:
            #print(n[0]['alt'])
            all1.append(n[0]['alt'])
        else:
            all1.append("unknown-product")

        if author is not None:
            #print(author.text)
            all1.append(author.text)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author.text)
            else:    
                all1.append('0')

        if rating is not None:
            #print(rating.text)
            all1.append(rating.text)
        else:
            all1.append('-1')

        if users_rated is not None:
            #print(price.text)
            all1.append(users_rated.text)
        else:
            all1.append('0')     

        if price is not None:
            #print(price.text)
            all1.append(price.text)
        else:
            all1.append('0')
        alls.append(all1)    
    return alls

In [12]:
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Book Name','Author','Rating','Customers_Rated', 'Price'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')

In [13]:
df = pd.read_csv("amazon_products.csv")

In [14]:
df.shape

(100, 5)

In [15]:
df.head(61)

,Book Name,Author,Rating,Customers_Rated,Price
0,The Psychology of Money,Morgan Housel,4.6 out of 5 stars,"18,818",₹277.00
1,400 Days,Chetan Bhagat,4.3 out of 5 stars,859,₹156.00
2,Atomic Habits: The life-changing million copy ...,James Clear,4.6 out of 5 stars,"26,324",₹475.00
3,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5 out of 5 stars,"38,119",₹349.00
4,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.6 out of 5 stars,"24,292",₹317.00
...,...,...,...,...,...
56,CBSE MCQ Bank Social Science Class 10 Term 1,www.cbse.online,4.2 out of 5 stars,25,₹389.00
57,General Knowledge,Dr. Binay Karna,4.4 out of 5 stars,"16,245",₹180.00
58,Mahatma Gandhi Autobiography: The Story Of My ...,Mahatma Gandhi,4.5 out of 5 stars,"4,897",₹99.00
59,Writing Practice Boxset: Pack of 4 Books (Writ...,Wonder House Books,4.3 out of 5 stars,"6,178",₹159.00
